# Visualização da Informação
## Escola de Matemática Aplicada - Fundação Getúlio Vargas
## Mestrado em Modelagem Matemática

Aluno: Gianlucca Devigili<br>
Github do projeto: 

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.express as px

df = pd.read_csv('world-happiness-report.csv')
df2021 = pd.read_csv('world-happiness-report-2021.csv')